# Evaluating job posts
This notebook provides sample code for conducting a text analysis using [EDSL](https://docs.expectedparrot.com), an open-source library for simulating surveys, experiments and other research with AI agents and large language models. 

Using a dataset of job posts as an example, we demonstrate how to: 

1. Import data into EDSL 
2. Create questions about the data 
3. Design an AI agent to answer the questions
4. Select a language model to generate responses
5. Analyze results as a formatted dataset


## Technical setup
Before running the code below please ensure that you have completed setup:

* [Install EDSL](https://docs.expectedparrot.com/en/latest/installation.html).
* Create a [Coop account](https://www.expectedparrot.com/login) and activate [remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html) OR store your own [API Keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for language models that you want to use.

Our [Starter Tutorial](https://docs.expectedparrot.com/en/latest/starter_tutorial.html) also provides examples of EDSL basic components. 

## Selecting data for review
First we identify some data for review. Data can be created using the EDSL tools or [imported from other sources](https://docs.expectedparrot.com/en/latest/scenarios.html). For purposes of this demo we import a set of job posts:

In [1]:
job_posts = [
    "Oversee daily operations, manage staff, and ensure customer satisfaction in a fast-paced retail environment.",
    "Craft engaging and informative blog posts on health and wellness topics to boost website traffic and engage readers.",
    "Analyze sales data using statistical tools to identify trends and provide actionable insights to the marketing team.",
    "Prepare gourmet dishes that comply with restaurant standards and delight customers with unique flavor combinations.",
    "Design creative visual content for marketing materials, including brochures, banners, and digital ads, using Adobe Creative Suite.",
    "Develop, test, and maintain robust software solutions to improve business processes using Python and Java.",
    "Craft coffee drinks and manage the coffee station while providing excellent customer service in a busy café.",
    "Manage recruitment processes, conduct interviews, and oversee employee benefit programs to ensure a motivated workforce.",
    "Assist veterinarians by preparing animals for surgery, administering injections, and providing post-operative care.",
    "Design aesthetic and practical outdoor spaces for clients, from residential gardens to public parks.",
    "Install and repair residential plumbing systems, including water heaters, pipes, and fixtures to ensure proper functionality.",
    "Develop comprehensive marketing strategies that align with company goals, including digital campaigns and branding efforts.",
    "Install, maintain, and repair electrical wiring, equipment, and fixtures to ensure safe and effective operation.",
    "Provide personalized fitness programs and conduct group fitness classes to help clients achieve their health goals.",
    "Diagnose and repair automotive issues, perform routine maintenance, and ensure vehicles meet safety standards.",
    "Lead creative campaigns, from concept through execution, coordinating with graphic designers and content creators.",
    "Educate students in mathematics using innovative teaching strategies to enhance understanding and interest in the subject.",
    "Drive sales through engaging customer interactions, understanding client needs, and providing product solutions.",
    "Fold dough into pretzel shapes ensuring each is uniformly twisted and perfectly salted before baking.",
    "Address customer inquiries and issues via phone and email, ensuring high levels of satisfaction and timely resolution.",
]

## Constructing questions about the data
Next we create some questions about the data. EDSL provides a variety of question types that we can choose from based on the form of the response that we want to get back from the model (multiple choice, free text, checkbox, linear scale, etc.). [Learn more about question types](https://docs.expectedparrot.com/en/latest/questions.html). 

Note that we use a `{{ placeholder }}` in each question text in order to parameterize the questions with the individual job posts in the next step:

In [2]:
from edsl import (
    QuestionList,
    QuestionLinearScale,
    QuestionMultipleChoice,
    QuestionYesNo,
    QuestionFreeText,
)

q1 = QuestionList(
    question_name="category_list",
    question_text="Draft a list of increasingly specific categories for the following job post: {{ job_post }}",
    max_list_items=3,  # optional
)

q2 = QuestionLinearScale(
    question_name="specific_scale",
    question_text="How specific is this job post: {{ job_post }}",
    question_options=[0, 1, 2, 3, 4, 5],
    option_labels={0: "Unclear", 1: "Not at all specific", 5: "Highly specific"},
)

q3 = QuestionMultipleChoice(
    question_name="skill_choice",
    question_text="What is the skill level required for this job: {{ job_post }}",
    question_options=["Entry level", "Intermediate", "Advanced", "Expert"],
)

q4 = QuestionYesNo(
    question_name="technical_yn",
    question_text="Is this a technical job? Job post: {{ job_post }}",
)

q5 = QuestionFreeText(
    question_name="rewrite_text",
    question_text="""Draft an improved version of the following job post: {{ job_post }}""",
)

## Building a survey
We combine the questions into a survey in order to administer them together:

In [3]:
from edsl import Survey

questions = [q1, q2, q3, q4, q5]

survey = Survey(questions)

If we want the agent/model to have information about prior questions in the survey we can add targeted or full memories ([learn more about adding survey rules/logic](https://docs.expectedparrot.com/en/latest/surveys.html)):

In [4]:
# Memory of a specific question is presented with another question:
# survey = survey.add_targeted_memory(q2, q1)

# Full memory of all prior questions is presented with each question (token-intensive):
# survey = survey.set_full_memory_mode()

## Adding data to the questions
We add the contents of each ticket into each question as an independent "scenario" for review. This allows us to create versions of the questions for each job post and deliver them to the model all at once. [EDSL provides many methods for generating scenarios](https://docs.expectedparrot.com/en/latest/scenarios.html) from different data sources (PDFs, CSVs, docs, images, tables, dicts, etc.). Here we import the list from above:

In [5]:
from edsl import ScenarioList, Scenario

scenarios = ScenarioList.from_list("job_post", job_posts)

## Designing AI agents
A key feature of EDSL is the ability to create personas for AI agents that the language models are prompted to use in generating responses to the questions. This is done by passing a dictionary of traits to Agent objects:

In [6]:
from edsl import AgentList, Agent

agent = Agent(traits={"persona":"You are a labor economist."})

## Selecting language models
EDSL allows us to select the language models to use in generating results. To see all available models:

In [7]:
from edsl import ModelList, Model

Model.available()

/Users/a16174/edsl/edsl/inference_services/AvailableModelFetcher.py:139: UserWarning: No models found for service ollama
  warnings.warn(f"No models found for service {service_name}")


,Model Name,Service Name
0,Austism/chronos-hermes-13b-v2,deep_infra
1,Gryphe/MythoMax-L2-13b,deep_infra
2,Qwen/Qwen2-72B-Instruct,deep_infra
3,Qwen/Qwen2-7B-Instruct,deep_infra
4,Qwen/Qwen2.5-72B-Instruct,deep_infra
5,Sao10K/L3-70B-Euryale-v2.1,deep_infra
6,Sao10K/L3.1-70B-Euryale-v2.2,deep_infra
7,google/gemma-2-27b-it,deep_infra
8,google/gemma-2-9b-it,deep_infra
9,lizpreciatior/lzlv_70b_fp16_hf,deep_infra


Here we select GPT 4o (if no model is specified, this model is also used by default):

In [8]:
model = Model("gpt-4o")

## Running the survey
We run the survey by adding the scenarios, agent and model with the `by()` method and then calling the `run()` method:

In [9]:
results = survey.by(scenarios).by(agent).by(model).run()

Job UUID,fb85ba30-31d9-401c-84b1-c920ac6fa5fc
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/fb85ba30-31d9-401c-84b1-c920ac6fa5fc
Error Report URL,None
Results UUID,b4c408ec-32d6-4e4a-8422-6fb9957c4b87
Results URL,None


This generates a dataset of `Results` that we can analyze with built-in methods for data tables, dataframes, SQL, etc. We can see a list of all the components that can be analyzed:

In [10]:
results.columns

,0
0,agent.agent_instruction
1,agent.agent_name
2,agent.persona
3,answer.category_list
4,answer.rewrite_text
5,answer.skill_choice
6,answer.specific_scale
7,answer.technical_yn
8,comment.category_list_comment
9,comment.rewrite_text_comment


For example, we can filter, sort, select, limit, shuffle, sample and print some components of results in a table:

In [11]:
(
    results
    .filter("specific_scale in [3,4,5]")
    .sort_by("skill_choice")
    .select(
        "model",
        "persona",
        "job_post",
        "category_list",
        "specific_scale",
        "skill_choice",
        "technical_yn",
    )
)

,model.model,agent.persona,scenario.job_post,answer.category_list,answer.specific_scale,answer.skill_choice,answer.technical_yn
0,gpt-4o,You are a labor economist.,"Develop, test, and maintain robust software solutions to improve business processes using Python and Java.","['Software Development', 'Business Process Automation', 'Python and Java Development']",3,Advanced,Yes
1,gpt-4o,You are a labor economist.,"Install and repair residential plumbing systems, including water heaters, pipes, and fixtures to ensure proper functionality.","['Plumbing', 'Residential Plumbing', 'Residential Plumbing Installation and Repair']",3,Advanced,Yes
2,gpt-4o,You are a labor economist.,Craft coffee drinks and manage the coffee station while providing excellent customer service in a busy café.,"['Food Service', 'Barista', 'Café Barista']",3,Entry level,No
3,gpt-4o,You are a labor economist.,"Design creative visual content for marketing materials, including brochures, banners, and digital ads, using Adobe Creative Suite.","['Graphic Design', 'Marketing Design', 'Digital and Print Marketing Design']",3,Intermediate,Yes
4,gpt-4o,You are a labor economist.,"Assist veterinarians by preparing animals for surgery, administering injections, and providing post-operative care.","['Animal Care', 'Veterinary Technician', 'Surgical Veterinary Technician']",5,Intermediate,Yes
5,gpt-4o,You are a labor economist.,"Install, maintain, and repair electrical wiring, equipment, and fixtures to ensure safe and effective operation.","['Electrical Work', 'Electrical Maintenance and Repair', 'Residential Electrical Maintenance and Repair']",3,Intermediate,Yes
6,gpt-4o,You are a labor economist.,"Install, maintain, and repair electrical wiring, equipment, and fixtures to ensure safe and effective operation.","['Electrical Work', 'Electrical Maintenance and Repair', 'Residential Electrical Maintenance and Repair']",3,Intermediate,Yes
7,gpt-4o,You are a labor economist.,Fold dough into pretzel shapes ensuring each is uniformly twisted and perfectly salted before baking.,"['Food Preparation', 'Baking', 'Pretzel Shaping']",5,Intermediate,No


In [12]:
results.select("rewrite_text")

,answer.rewrite_text
0,"Job Title: Retail Operations Manager Job Description: We are seeking a dynamic and experienced Retail Operations Manager to lead our team in delivering exceptional service and operational excellence in a fast-paced retail environment. The successful candidate will be responsible for overseeing daily operations, managing and mentoring staff, and ensuring the highest levels of customer satisfaction. Key Responsibilities: - Lead and manage daily store operations to ensure efficiency and effectiveness. - Inspire, train, and develop a talented team of retail staff to achieve performance goals. - Foster a customer-centric culture by ensuring all team members deliver outstanding service. - Monitor inventory levels and implement strategies to optimize stock management. - Analyze sales data and customer feedback to drive continuous improvement. - Ensure compliance with company policies and retail regulations. Qualifications: - Proven experience in retail management or a similar role. - Strong leadership and team-building skills. - Excellent communication and interpersonal abilities. - Ability to thrive in a fast-paced, dynamic environment. - Exceptional problem-solving and decision-making skills."
1,"We are seeking a talented and passionate writer to create captivating and insightful blog posts on health and wellness topics. Your work will play a crucial role in enhancing our website's visibility and fostering a strong connection with our audience. If you have a knack for crafting compelling content that informs and inspires, we would love to hear from you!"
2,"Join our dynamic team as a Sales Data Analyst! In this role, you will leverage advanced statistical tools to meticulously examine sales data, uncovering key trends that drive our business forward. Your expert insights will directly empower our marketing team to make informed, strategic decisions. If you have a passion for data and a knack for translating numbers into actionable strategies, we want to hear from you!"
3,"Join our dynamic team as a Sales Data Analyst! In this role, you will leverage advanced statistical tools to meticulously examine sales data, uncovering key trends that drive our business forward. Your expert insights will directly empower our marketing team to make informed, strategic decisions. If you have a passion for data and a knack for translating numbers into actionable strategies, we want to hear from you!"
4,"Join our culinary team as a Gourmet Chef! We are seeking a talented and passionate individual to craft exquisite dishes that not only meet our high restaurant standards but also captivate our guests with innovative and delightful flavor combinations. If you have a flair for creativity and a commitment to excellence, we would love to hear from you."
5,"Join our dynamic marketing team as a Creative Designer! We are looking for a talented individual to craft visually compelling content across a variety of platforms. Your primary responsibilities will include designing innovative marketing materials such as brochures, banners, and digital advertisements. Proficiency in Adobe Creative Suite is essential. If you have a keen eye for detail and a passion for creativity, we would love to see your work!"
6,"Job Title: Software Developer - Python & Java Job Description: Join our dynamic team as a Software Developer, where you will play a crucial role in enhancing our business processes through innovative software solutions. We are looking for a talented and motivated individual who is proficient in Python and Java to develop, test, and maintain cutting-edge applications that drive efficiency and growth. Key Responsibilities: - Design and implement robust software solutions tailored to improve business operations. - Collaborate with cross-functional teams to identify and address process inefficiencies. - Develop and maintain high-quality code using Python and Java. - Conduct thorough testing to ensure the reliability and performance of s

## Posting content to the Coop
We can post any objects to the Coop, including this notebook. Objects can be updated or modified at your Coop account, and shared with others or stored privately (default visibility is *unlisted*):

In [13]:
survey.push(description = "Example survey: Job posts analysis", visibility = "public")

{'description': 'Example survey: Job posts analysis',
 'object_type': 'survey',
 'url': 'https://www.expectedparrot.com/content/1563e393-6db3-430c-b971-a7b295812249',
 'uuid': '1563e393-6db3-430c-b971-a7b295812249',
 'version': '0.1.39.dev2',
 'visibility': 'public'}

In [14]:
from edsl import Notebook

n = Notebook(path = "evaluating_job_posts.ipynb")

In [15]:
info = n.push(description = "Example text analysis: evaluating job posts", visibility = "public")
info

{'description': 'Example text analysis: evaluating job posts',
 'object_type': 'notebook',
 'url': 'https://www.expectedparrot.com/content/6759d869-5734-4c69-8262-df6038179aca',
 'uuid': '6759d869-5734-4c69-8262-df6038179aca',
 'version': '0.1.39.dev2',
 'visibility': 'public'}

To update an object at the Coop:

In [16]:
n = Notebook(path = "evaluating_job_posts.ipynb") # resave the object

n.patch(uuid = info["uuid"], value = n)

{'status': 'success'}